In [66]:
import torch
import pandas as pd
from sklearn.linear_model import LogisticRegression
import pickle

In [6]:
voxlingua_embed_path='/Users/markjos/projects/malachor5/data/tira-drz/tira-sli-voxlingua.pt'
commonvoice_embed_path='/Users/markjos/projects/malachor5/data/tira-drz/tira-sli-commonlang.pt'
metadata='/Users/markjos/projects/malachor5/data/tira-drz/tira-sli-voxlingua.csv'

In [17]:
vl_embeds=torch.load(voxlingua_embed_path)
cv_embeds=torch.load(commonvoice_embed_path)
df=pd.read_csv(metadata)
vl_embeds['train'].shape, cv_embeds['train'].shape, df.shape

(torch.Size([7274, 1, 256]), torch.Size([7274, 1, 256]), (9092, 14))

In [28]:
vl_train_X=vl_embeds['train'].squeeze()
vl_test_X=vl_embeds['test'].squeeze()
cv_train_X=cv_embeds['train'].squeeze()
cv_test_X=cv_embeds['test'].squeeze()
vl_train_X.shape, cv_train_X.shape, vl_test_X.shape, vl_test_X.shape,

(torch.Size([7274, 256]),
 torch.Size([7274, 256]),
 torch.Size([1818, 256]),
 torch.Size([1818, 256]))

In [29]:
train_Y=df.loc[df['split']=='train', 'label']
test_Y=df.loc[df['split']=='test', 'label']
train_Y.shape, test_Y.shape

((7274,), (1818,))

In [30]:
train_Y.value_counts(), test_Y.value_counts()

(label
 0    3637
 1    3637
 Name: count, dtype: int64,
 label
 0    909
 1    909
 Name: count, dtype: int64)

In [63]:
vl_LR=LogisticRegression(max_iter=500).fit(vl_train_X, train_Y)
cv_LR=LogisticRegression(max_iter=500).fit(cv_train_X, train_Y)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

In [64]:
vl_LR.score(vl_test_X, test_Y)

0.9543454345434543

In [65]:
cv_LR.score(cv_test_X, test_Y)

0.9543454345434543

In [69]:
lr_path='/Users/markjos/projects/malachor5/data/tira-drz/commonlang_logreg.pkl'

with open(lr_path, 'wb') as f:
    pickle.dump(cv_LR, f)

Test loading model and doing inference

In [72]:
with open(lr_path, 'rb') as f:
    unpkld = pickle.load(f)
unpkld

LogisticRegression(max_iter=500)

In [78]:
unpkld.score(cv_test_X, test_Y)

0.9543454345434543

In [76]:
unpkld.predict(cv_test_X)

array([0, 0, 0, ..., 1, 1, 1])